In [ ]:
!pip install --upgrade geopandas
!pip install folium
!pip install mapclassify
!pip install geopy
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import folium
import geopy.distance
import math

In [ ]:
!pip install georasters

In [33]:
mapa_judete = gpd.read_file('/content/drive/MyDrive/EduHack 2023/Geodatabases/ro_judete_poligon.geojson')
mapa_timis = mapa_judete[mapa_judete["name"] == "Timiș"]

In [35]:
timisoara_trash = pd.read_csv("/content/drive/MyDrive/EduHack 2023/Geodatabases/tiisoara_trash_cans.csv")
geo_timisoara_trash = gpd.GeoDataFrame(timisoara_trash, geometry=gpd.GeoSeries.from_xy(timisoara_trash['longitudine'], timisoara_trash['latitudine']), crs=4326)

geo_timisoara_trash.columns = ['_id', 'id','Type', 'adresa', 'companie', 'website',
       'latitudine', 'longitudine', 'count', 'geometry']

trash_types_original = ['sticlă', 'medicamente expirate', 'ulei utilizat',
       'carton, hârtie, plastic',
       'doze aluminiu, baterii, sticlă, becuri, hârtie, plastic',
       'baterii, ulei de mașină utilizat, ulei utilizat', 'haine',
       'doze aluminiu, baterii, sticlă, becuri, plastic', 'baterii',
       'baterii, sticlă, hârtie',
       'aparate electrocasnice mici, baterii, becuri',
       'carton, polistiren, deșeuri voluminoase',
       'doze aluminiu, aparate electrocasnice mici, baterii, sticlă, becuri, hârtie, plastic',
       'aparate electrocasnice mici, baterii, sticlă, becuri, hârtie, plastic']
trash_types_converted = ['glass', 'expired medicines', 'used oil',
'cardboard, paper, plastic',
'aluminum cans, batteries, glass, bulbs, paper, plastic',
'batteries, used motor oil, used oil', 'clothing',
'aluminum cans, batteries, glass, bulbs, plastic', 'batteries',
'batteries, glass, paper',
'small appliances, batteries, bulbs',
'cardboard, polystyrene, bulky waste',
'aluminum cans, small appliances, batteries, glass, bulbs, paper, plastic',
'small appliances, batteries, glass, bulbs, paper, plastic']

type_dict = dict(zip(trash_types_original,trash_types_converted))
geo_timisoara_trash=geo_timisoara_trash.replace({"Type": type_dict})

In [36]:
pop_1km_2018 = gpd.read_file('/content/drive/MyDrive/EduHack 2023/Geodatabases/Pop_counts_1km/pop_1km_2018_final.geojson')


In [ ]:
#pop_1km_2018 = gpd.read_file('/content/drive/MyDrive/EduHack 2023/Geodatabases/Pop_counts_1km/2018_limpo/JRC_POPULATION_2018.shp')

In [ ]:
pop_1km_2018

,OBJECTID,GRD_ID,CNTR_ID,TOT_P_2018,Country,Date,Method,Shape_Leng,Shape_Area,geometry
0,517287,CRS3035RES1000mN2396000E5364000,RO,10,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5364000.000 2396000.000, 5364000.000..."
1,517288,CRS3035RES1000mN2396000E5365000,RO,9,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5365000.000 2396000.000, 5365000.000..."
2,517289,CRS3035RES1000mN2396000E5366000,RO,78,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5366000.000 2396000.000, 5366000.000..."
3,517321,CRS3035RES1000mN2396000E5465000,BG-RO,3,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5465000.000 2396000.000, 5465000.000..."
4,518678,CRS3035RES1000mN2397000E5363000,RO,135,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5363000.000 2397000.000, 5363000.000..."
...,...,...,...,...,...,...,...,...,...,...
71210,1203558,CRS3035RES1000mN2932000E5557000,MD-RO,97,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5557000.000 2932000.000, 5557000.000..."
71211,1203559,CRS3035RES1000mN2932000E5558000,MD-RO,93,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5558000.000 2932000.000, 5558000.000..."
71212,1204767,CRS3035RES1000mN2933000E5556000,MD-RO,3,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5556000.000 2933000.000, 5556000.000..."
71213,1204768,CRS3035RES1000mN2933000E5557000,MD-RO,90,RO,2018-01-01,Disaggregation of 2011-2018 changes,4000.0,1000000.0,"POLYGON ((5557000.000 2933000.000, 5557000.000..."


In [37]:
#pop_1km_2018 = gpd.read_file('/content/drive/MyDrive/EduHack 2023/Geodatabases/Pop_counts_1km/2018_limpo/JRC_POPULATION_2018.shp')
forecast_output = pd.read_excel("/content/drive/MyDrive/EduHack 2023/Geodatabases/Pop_counts_1km/romania_1km_population_forecast.xlsx")


pop_1km_2018 = pop_1km_2018.merge(forecast_output,on="GRD_ID",how="left")

mapa_timis = mapa_timis.to_crs(4326)
pop_1km_2018 = pop_1km_2018.to_crs(4326)
pop_1km_2018_timis = pop_1km_2018.sjoin(mapa_timis,how="inner")

In [38]:
map = folium.Map(location= [45.760696,21.226788],prefer_canvas=True,tiles="CartoDB Positron", zoom_start=12)


# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_timisoara_trash.geometry]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    # assign a color marker for the type of volcano, Strato being the most common
    if geo_timisoara_trash.Type[i] == "glass":
        type_color = "green"
    elif geo_timisoara_trash.Type[i] == "expired medicines":
        type_color = "pink"
    elif geo_timisoara_trash.Type[i] == "used oil":
        type_color = "red"
    elif geo_timisoara_trash.Type[i] == "cardboard, paper, plastic":
        type_color = "green"
    elif geo_timisoara_trash.Type[i] == "aluminum cans, batteries, glass, bulbs, paper, plasti":
        type_color = "yellow"
    elif geo_timisoara_trash.Type[i] == "batteries, used motor oil, used oil":
        type_color = "red"
    elif geo_timisoara_trash.Type[i] == "clothing":
        type_color = "beige"
    elif geo_timisoara_trash.Type[i] == "aluminum cans, batteries, glass, bulbs, plastic":
        type_color = "yellow"
    elif geo_timisoara_trash.Type[i] == "batteries, glass, paper":
        type_color = "pink"
    elif geo_timisoara_trash.Type[i] == "small appliances, batteries, bulbs":
        type_color = "yellow"
    elif geo_timisoara_trash.Type[i] == 'cardboard, polystyrene, bulky waste':
        type_color = "blue"
    elif geo_timisoara_trash.Type[i] == 'aluminum cans, small appliances, batteries, glass, bulbs, paper, plastic':
        type_color = "black"
    else:
        type_color = "darkpurple"

    # Place the markers with the popup labels and data
    map.add_child(
        folium.Marker(
            location=coordinates,
            popup="Type: "
            + str(geo_timisoara_trash.Type[i])
            + "<br>"
            + "Company: "
            + str(geo_timisoara_trash.companie[i])
            + "<br>"
            + "Adress: "
            + str(geo_timisoara_trash.adresa[i])
            + "<br>"
            + "Coordinates: "
            + str(geo_timisoara_trash.geometry[i]),
            icon=folium.Icon(color=type_color,fill_opacity=0.4,radius=0.5,icon='trash'),
        )
    )
    i = i + 1

folium.Choropleth(
    geo_data=pop_1km_2018_timis,
    name="choropleth",
    data=pop_1km_2018_timis,
    columns=["GRD_ID","TOT_P_2018_x"],
    key_on="feature.properties.GRD_ID",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="GRD_ID",
).add_to(map)

folium.LayerControl().add_to(map)


<ipython-input-38-c82c101be5ae>:53: UserWarning: color argument of Icon should be one of: {'lightgray', 'cadetblue', 'darkgreen', 'pink', 'blue', 'green', 'beige', 'orange', 'lightgreen', 'black', 'white', 'red', 'darkpurple', 'gray', 'lightred', 'lightblue', 'darkblue', 'purple', 'darkred'}.
  icon=folium.Icon(color=type_color,fill_opacity=0.4,radius=0.5,icon='trash'),


In [42]:
pop_1km_2018_timis = pop_1km_2018_timis.to_crs(4326)
preserve_geometry = pop_1km_2018_timis[["GRD_ID","geometry"]]

preserve_geometry.columns = ["GRD_ID","geometry_v2"]

In [ ]:
# CRS for distance calculation in meters
pop_1km_2018_timis = pop_1km_2018_timis.to_crs(3035)
geo_timisoara_trash = geo_timisoara_trash.to_crs(3035)

In [ ]:
# calculate sum, average and distances from every grid to the N closest trash cans
n_closest_neighbors = 5
average_distance_to_n_closest_neighbors = []
distance_to_n_closest_neighbors = []
sum_distance_to_n_closest_neighbors = []
for population_grid in pop_1km_2018_timis.itertuples():
  distances = []
  for trash_can in geo_timisoara_trash.itertuples():
    distances.append(pop_1km_2018_timis['geometry'][population_grid.Index].distance(geo_timisoara_trash['geometry'][trash_can.Index]))
  distances = np.sort(distances)[0:n_closest_neighbors]
  distance_to_n_closest_neighbors.append([pop_1km_2018_timis['GRD_ID'][population_grid.Index],distances])
  average_distance_to_n_closest_neighbors.append([pop_1km_2018_timis['GRD_ID'][population_grid.Index],np.average(distances)])
  sum_distance_to_n_closest_neighbors.append([pop_1km_2018_timis['GRD_ID'][population_grid.Index],np.sum(distances)])


In [ ]:
avg_distance = pd.DataFrame(average_distance_to_n_closest_neighbors)
avg_distance.columns = ["GRD_ID","average_distance"]
sum_distance = pd.DataFrame(sum_distance_to_n_closest_neighbors)
sum_distance.columns = ["GRD_ID","sum_distance"]

pop_1km_2018_timis = pop_1km_2018_timis.merge(avg_distance, on='GRD_ID',how='left')

pop_1km_2018_timis = pop_1km_2018_timis.merge(sum_distance, on='GRD_ID',how='left')


In [ ]:
#pop_1km_2018_timis = pop_1km_2018_timis[['GRD_ID', 'TOT_P_2018_x','geometry', 'pop2019', 'pop2020', 'pop2021',
 #      'pop2022', 'pop2023', 'pop2024', 'pop2025', 'pop2026', 'pop2027',
#       'pop2028', 'countyId', 'countyCode', 'name']]
#geo_timisoara_trash = geo_timisoara_trash[[ 'id', 'Type', 'adresa', 'companie',
  #     'website', 'latitudine', 'longitudine', 'count', 'geometry']]

In [ ]:
# CRS for distance calculation in meters
pop_1km_2018_timis = pop_1km_2018_timis.to_crs(3035)
geo_timisoara_trash = geo_timisoara_trash.to_crs(3035)

In [ ]:
pop_1km_2018_timis = pop_1km_2018_timis.drop("index_right",axis=1)


In [ ]:
#buffer_size_big = 20

buffer_size_big = 7

#create big buffer size and remove all datapoints with no trash can nearby to remove OUTLIERS

#create smaller buffer size and perform final calulations

#number of trash cans in X meters / population * avg distance to 10 nearest trash cans * normalizer

In [ ]:
lamb = pop_1km_2018_timis
pop_20km_buffer = pop_1km_2018_timis
pop_20km_buffer.geometry = pop_1km_2018_timis.buffer(20000)
pop_20km_buffer = pop_20km_buffer[['GRD_ID',"geometry"]]
pop_20km_buffer = pop_20km_buffer.merge(
    lamb.sjoin(
        geo_timisoara_trash
    ).groupby(
        'GRD_ID'
    ).count().geometry.rename(
        'n_points'
    ).reset_index())["GRD_ID"].values

pop_1km_2018_timisoara = pop_1km_2018_timis[pop_1km_2018_timis["GRD_ID"].isin(pop_20km_buffer)]

lamb_2 = pop_1km_2018_timisoara
pop_6km_buffer = pop_1km_2018_timisoara
pop_6km_buffer.geometry = pop_1km_2018_timis.buffer(6000)
pop_6km_buffer = pop_6km_buffer[['GRD_ID',"geometry"]]
pop_6km_buffer = pop_6km_buffer.merge(
    lamb_2.sjoin(
        geo_timisoara_trash
    ).groupby(
        'GRD_ID'
    ).count().geometry.rename(
        'n_trash_cans_in_buffer'
    ).reset_index())
pop_6km_buffer = pop_6km_buffer[["GRD_ID",'n_trash_cans_in_buffer']]

pop_1km_2018_timisoara = pop_1km_2018_timisoara.merge(pop_6km_buffer,on="GRD_ID",how="left")

pop_1km_2018_timisoara = pop_1km_2018_timisoara.to_crs(4326)
geo_timisoara_trash = geo_timisoara_trash.to_crs(4326)


/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
pop_1km_2018_timisoara = pop_1km_2018_timisoara[['GRD_ID','TOT_P_2018_x','geometry',
       2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028,
       'countyId','name','average_distance',
       'sum_distance', 'n_trash_cans_in_buffer']]

pop_1km_2018_timisoara.columns = ['GRD_ID','TOT_P_2018_x','geometry',
       "pop2019", "pop2020", "pop2021", "pop2022", "pop2023", "pop2024", "pop2025", "pop2026", "pop2027", "pop2028",
       'countyId','name','average_distance',
       'sum_distance', 'n_trash_cans_in_buffer']

pop_1km_2018_timisoara = pop_1km_2018_timisoara[pop_1km_2018_timisoara["pop2028"] != 0]

In [ ]:
from sklearn.preprocessing import StandardScaler

pop_1km_2018_timisoara[['n_trash_cans_in_buffer_scaled', 'pop2028_scaled', 'sum_distance_scaled','average_distance_scaled']]= StandardScaler().fit_transform(pop_1km_2018_timisoara[['n_trash_cans_in_buffer', 'pop2028', 'sum_distance','average_distance']])


pop_1km_2018_timisoara["recycling_acessibility_index_A"] =  pop_1km_2018_timisoara['n_trash_cans_in_buffer'] / pop_1km_2018_timisoara["pop2028"]

pop_1km_2018_timisoara["recycling_acessibility_index_B"] =  pop_1km_2018_timisoara["pop2028"] / (pop_1km_2018_timisoara['sum_distance'] / 1000 )

pop_1km_2018_timisoara["recycling_acessibility_index_C"] =  pop_1km_2018_timisoara["pop2028"] / (pop_1km_2018_timisoara['average_distance'] / 1000)

pop_1km_2018_timisoara["recycling_acessibility_index_D"] =  pop_1km_2018_timisoara['n_trash_cans_in_buffer'] / pop_1km_2018_timisoara["pop2028"] * pop_1km_2018_timisoara['sum_distance_scaled']

In [ ]:
folium.Choropleth(
    geo_data=pop_1km_2018_timisoara,
    name="choropleth",
    data=pop_1km_2018_timisoara,
    columns=["GRD_ID","recycling_acessibility_index_A"],
    key_on="feature.properties.GRD_ID",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="GRD_ID",
).add_to(map)

In [ ]:
preserve_geometry = preserve_geometry[preserve_geometry["GRD_ID"].isin(pop_1km_2018_timisoara["GRD_ID"].values)]

pop_1km_2018_timisoara = pop_1km_2018_timisoara.merge(preserve_geometry,on="GRD_ID",how="left")
pop_1km_2018_timisoara = pop_1km_2018_timisoara.drop("geometry",axis=1)
pop_1km_2018_timisoara.set_geometry("geometry_v2")
pop_1km_2018_timisoara.columns = ['GRD_ID', 'TOT_P_2018_x', 'pop2019', 'pop2020', 'pop2021', 'pop2022',
       'pop2023', 'pop2024', 'pop2025', 'pop2026', 'pop2027', 'pop2028',
       'countyId', 'name', 'average_distance', 'sum_distance',
       'n_trash_cans_in_buffer', 'n_trash_cans_in_buffer_scaled',
       'pop2028_scaled', 'sum_distance_scaled', 'average_distance_scaled',
       'recycling_acessibility_index_A', 'recycling_acessibility_index_B',
       'recycling_acessibility_index_C', 'recycling_acessibility_index_D',
       'geometry']

In [59]:
pop_1km_2018_timisoara = pop_1km_2018_timisoara[['GRD_ID', 'TOT_P_2018_x', 'pop2019', 'pop2020', 'pop2021', 'pop2022',
       'pop2023', 'pop2024', 'pop2025', 'pop2026', 'pop2027', 'pop2028',
       'countyId', 'name', 'average_distance', 'sum_distance',
       'n_trash_cans_in_buffer', 'n_trash_cans_in_buffer_scaled',
       'pop2028_scaled', 'sum_distance_scaled', 'average_distance_scaled',
       'recycling_acessibility_index_A', 'recycling_acessibility_index_B',
       'recycling_acessibility_index_C', 'recycling_acessibility_index_D',
       'geometry_v2_x']]

pop_1km_2018_timisoara.columns = ['GRD_ID', 'TOT_P_2018_x', 'pop2019', 'pop2020', 'pop2021', 'pop2022',
       'pop2023', 'pop2024', 'pop2025', 'pop2026', 'pop2027', 'pop2028',
       'countyId', 'name', 'average_distance', 'sum_distance',
       'n_trash_cans_in_buffer', 'n_trash_cans_in_buffer_scaled',
       'pop2028_scaled', 'sum_distance_scaled', 'average_distance_scaled',
       'recycling_acessibility_index_A', 'recycling_acessibility_index_B',
       'recycling_acessibility_index_C', 'recycling_acessibility_index_D',
       'geometry']

In [64]:
pop_1km_2018_timisoara = gpd.GeoDataFrame(pop_1km_2018_timisoara, crs="EPSG:4326", geometry="geometry")
pop_1km_2018_timisoara.to_file("export_final_v4.geojson",driver="GeoJSON")
geo_timisoara_trash.to_file("geo_timisoara_trash_v4.geojson",driver="GeoJSON")